In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/hw4

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/hw4


In [2]:
!pip install torch
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.6 MB/s eta 0:00:00


In [3]:
import torch
from transformers import GPT2Tokenizer, AutoTokenizer, GPT2LMHeadModel
from datasets import load_dataset
from collections import defaultdict
import copy

## **Task 1: Byte pair encoding**

The goal is to calculate byte pair encodings for a given dataset. For a recap of the concept we refer to lecture 9, slides 22.ff and internet search.

### Task 1.1
First we want to make sure we create a suffieciently large corpus. As dataset you import the dataset "wikitext-2-raw-v1" from huggingface. Make sure to use the 500 first entries of the training dataset and subsequently filter out empty  entries. Also print the first entry of the corpus.

In [4]:
# Load WikiText-2 dataset "wikitext-2-raw-v1"
wiki_dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
train_dataset = wiki_dataset["train"]
corpus = []
# YOUR CODE HERE
#iterate over the first 500 entries of the trainingset and filter out empty entries
for sent in train_dataset["text"][:500]:
  if sent not in ["", " "]:
    corpus.append(sent)

print(f"First Entry == {corpus[0]}")

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Dataset wikitext downloaded and prepared to /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

First Entry ==  = Valkyria Chronicles III = 



In [5]:
corpus

[' = Valkyria Chronicles III = \n',
 ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n',
 " The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving f

Note that the corpus should have a format like
corpus = [
    "This is the first sentence",
    "This is the second sentence",
    ..
]. For debugging during coding you can also consider working with a smaller corpus, e.g. only 5 sentences

### Task 1.2
Here, you loop through the corpus and count the word frequencies

In [94]:
# Initialize a tokenizer from the transformers library
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Initialize a defaultdict to store word frequencies
word_freqs = defaultdict(int)
# this dictionary gets entries for word_freqs[word]

alphabet = []
vocab = [""]
splits = {}
vocab_size = 200
merges = {}

# We loop through to corups to calculate word frequencies
for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    # YOUR CODE HERE to count word frequencies
    #word_freqs[]
    #print(words_with_offsets)
    #print(f"NEW_WORDS == {new_words}")
    for w in new_words:
      word_freqs[w] += 1

In [95]:
#dict(sorted(word_freqs.items(), key=lambda x: x[1], reverse=True))

The following block creates the alphabet and initial vocabulary

In [96]:
aa = ["b", "A", "a", "L"]
aa.sort()

In [97]:
aa

['A', 'L', 'a', 'b']

In [98]:
for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()

print("alphabet",alphabet)
vocab = ["<|endoftext|>"] + alphabet.copy()
splits = {word: [c for c in word] for word in word_freqs.keys()}

alphabet ['!', '"', '$', '%', '&', "'", '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '~', '¡', '¢', '£', '¤', '¥', '¦', '©', 'ª', '«', '®', '¯', '°', '±', '²', '³', '´', 'µ', '¹', '¼', '½', '¿', 'Â', 'Ã', 'Å', 'â', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ï', 'Ċ', 'Ġ', 'Ģ', 'ģ', 'Ĥ', 'ĥ', 'Ħ', 'ħ', 'Ĩ', 'Ī', 'ī', 'Ĭ', 'ĭ', 'Į', 'į', 'ı', 'Ĳ', 'ĳ', 'Ĵ', 'ĵ', 'Ķ', 'Ĺ', 'ĺ', 'Ļ', 'Ľ', 'Ł', 'ł', 'Ń']


In [99]:
"".join(splits['Ġfelt']) == 'Ġfelt'

True

In [100]:
list(splits.items())[0]

('Ġ=', ['Ġ', '='])

### Task 1.3
Define a function to compute the frequency of each pair of characters

In [101]:
# Define a function to compute the frequency of each pair of characters
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    # YOUR CODE HERE to compute the frequency of each pair of characters
    for k, v in splits.items():
      #v is a list of characters
      for i in range(len(v) - 1):
        pair = (v[i], v[i+1])
        pair_freqs[pair] += 1
    return pair_freqs

# Initialize pair_freqs by calling the compute_pair_freqs function
pair_freqs = compute_pair_freqs(splits)

In [102]:
sorted(pair_freqs.items(), key=lambda x: x[1], reverse=True)[:50]

[(('i', 'n'), 582),
 (('e', 'r'), 525),
 (('e', 'd'), 522),
 (('o', 'n'), 467),
 (('t', 'e'), 422),
 (('r', 'e'), 420),
 (('t', 'i'), 401),
 (('e', 's'), 377),
 (('e', 'n'), 365),
 (('Ġ', 's'), 362),
 (('n', 'g'), 352),
 (('a', 't'), 329),
 (('a', 'n'), 322),
 (('a', 'l'), 311),
 (('o', 'r'), 305),
 (('s', 't'), 301),
 (('a', 'r'), 296),
 (('Ġ', 'c'), 283),
 (('n', 't'), 275),
 (('Ġ', 'p'), 269),
 (('d', 'e'), 252),
 (('l', 'e'), 244),
 (('s', 'e'), 241),
 (('r', 'i'), 235),
 (('i', 'o'), 230),
 (('r', 'a'), 230),
 (('Ġ', 'a'), 224),
 (('c', 'o'), 208),
 (('i', 't'), 208),
 (('i', 's'), 205),
 (('Ġ', 'r'), 199),
 (('v', 'e'), 195),
 (('i', 'c'), 191),
 (('n', 'e'), 188),
 (('Ġ', 'd'), 186),
 (('r', 'o'), 183),
 (('l', 'i'), 179),
 (('e', 'l'), 179),
 (('m', 'e'), 177),
 (('e', 'a'), 173),
 (('l', 'a'), 169),
 (('n', 's'), 164),
 (('s', 'i'), 163),
 (('Ġ', 'm'), 162),
 (('n', 'd'), 160),
 (('Ġ', 't'), 159),
 (('t', 'a'), 159),
 (('c', 'e'), 158),
 (('l', 'l'), 157),
 (('Ġ', 'f'), 155)]

Here we find the pair of characters that appears most frequently together

In [103]:
for i, key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")
    if i >= 5:
        break

best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print("\nMost frequent pair of characters : ", best_pair, " occurs ", max_freq, " times")

('Ġ', '='): 1
('Ġ', 'V'): 29
('V', 'a'): 10
('a', 'l'): 311
('l', 'k'): 9
('k', 'y'): 5

Most frequent pair of characters :  ('i', 'n')  occurs  582  times


### Task 1.4
Define a function to merge the most frequent pair of characters

In [104]:
def help_fct(a, b, split):
  merged_list = []
  i = 0
  while i < len(split):
    if split[i] == a and i + 1 < len(split) and split[i + 1] == b:
      merged_list.append(a + b)
      i += 2
    else:
      merged_list.append(split[i])
      i += 1

  return merged_list

In [105]:
#tests
lst = ['c', 'a', 'b', 'd', 'e', 'f', 'a', 'b', 'g', 'h']
lst2 = "e n t e r t a i n i n g".split(" ")
lst3 = ["kee", "en", "ter", "z", "bl", "oo"]
print(help_fct(a="i", b="n", split=lst2))
print(help_fct(a="en", b="ter", split=lst3))

['e', 'n', 't', 'e', 'r', 't', 'a', 'in', 'in', 'g']
['kee', 'enter', 'z', 'bl', 'oo']


In [106]:

# Define a function to merge the most frequent pair of characters
def merge_pair(a, b, splits):
    #splits_copy = copy.deepcopy(splits)
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue

        # YOUR CODE HERE to merge the most frequent pair of characters in each word
        splits[word] = help_fct(a=a, b=b, split=split)
    return splits

In [107]:
splits["Ġserving"]

['Ġ', 's', 'e', 'r', 'v', 'i', 'n', 'g']

In [108]:
merge_pair(a="i", b="n", splits=splits)["Ġserving"]

['Ġ', 's', 'e', 'r', 'v', 'in', 'g']

### Task 1.5

Iterate this process until you reach a predefined vocabulary size

In [109]:
# YOUR CODE HERE to iterate this process until you reach a predefined vocabulary size
#s_final = dict()
# Edited my this part of my code based on the following link: https://huggingface.co/learn/nlp-course/chapter6/5?fw=pt
vocab_size = 200
merges = {("i", "n"): "in"}
vocab.append("in")
while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits = merge_pair(*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

In [110]:
print(merges)

{('i', 'n'): 'in', ('e', 'r'): 'er', ('e', 'd'): 'ed', ('o', 'n'): 'on', ('e', 's'): 'es', ('e', 'n'): 'en', ('Ġ', 's'): 'Ġs', ('a', 't'): 'at', ('a', 'n'): 'an', ('a', 'l'): 'al', ('o', 'r'): 'or', ('a', 'r'): 'ar', ('in', 'g'): 'ing', ('Ġ', 'c'): 'Ġc', ('Ġ', 'p'): 'Ġp', ('r', 'e'): 're', ('t', 'i'): 'ti', ('i', 's'): 'is', ('Ġ', 'd'): 'Ġd', ('e', 'l'): 'el', ('Ġ', 'm'): 'Ġm', ('Ġ', 'f'): 'Ġf', ('i', 'c'): 'ic', ('i', 't'): 'it', ('Ġ', 't'): 'Ġt', ('o', 'u'): 'ou', ('l', 'e'): 'le', ('Ġ', 'a'): 'Ġa', ('r', 'o'): 'ro', ('Ġ', 'S'): 'ĠS', ('i', 'on'): 'ion', ('Ġ', 'b'): 'Ġb', ('en', 't'): 'ent', ('Ġ', 'C'): 'ĠC', ('Ġ', 're'): 'Ġre', ('Ġ', '1'): 'Ġ1', ('i', 'l'): 'il', ('o', 'm'): 'om', ('a', 's'): 'as', ('a', 'c'): 'ac', ('e', 'c'): 'ec', ('Ġ', 'w'): 'Ġw', ('Ġ', 'A'): 'ĠA', ('l', 'o'): 'lo', ('s', 't'): 'st', ('l', 'y'): 'ly', ('Ġ', 'e'): 'Ġe', ('Ġ', 'M'): 'ĠM', ('e', 't'): 'et', ('Ġ', 'in'): 'Ġin', ('Ġ', 'P'): 'ĠP', ('Ġ', 'h'): 'Ġh', ('u', 'r'): 'ur', ('er', 's'): 'ers', ('r', 'a'): 'ra

In [111]:
print(len(merges))

56


In [112]:
len(vocab)

200

In [ ]:
# YOUR CODE HERE to iterate this process until you reach a predefined vocabulary size
"""s_final = dict()
for idx, elem in enumerate(sorted(pair_freqs.items(), key=lambda x: x[1], reverse=True)):
  if idx == 0:
    s_final = merge_pair(a=elem[0][0], b=elem[0][1], splits=splits)
  else:
    s_final = merge_pair(a=elem[0][0], b=elem[0][1], splits=s_final)"""

In [21]:
"""def get_vocab(vocab_size=200, splits=splits):
  counter_gl = 0
  final_vocab = dict()
  sp_final = copy.deepcopy(splits)
  while counter_gl < vocab_size:
    counter_loc = 0
    pair_freqs_sp_final = compute_pair_freqs(sp_final)
    most_freq_a, most_freq_b = sorted(pair_freqs_sp_final.items(), key=lambda x: x[1], reverse=True)[0][0]
    sp_final = merge_pair(a=most_freq_a, b=most_freq_b, splits=sp_final)
    #count built vocabulary
    for elem in list(sp_final.items()):
      if len(elem[1]) == 1:
        final_vocab[elem[0]] = elem[1]
        counter_loc += 1
        if counter_loc == 200:
          #print(counter_loc)
          break
    if counter_loc == 200:
      counter_gl = counter_loc
      print("counter_gl set to 200")

  return sp_final, final_vocab"""

In [22]:
"""dict_merged, final_vocab = get_vocab(vocab_size=vocab_size)"""

counter_gl set to 200


In [23]:
"""final_vocab.keys()"""

dict_keys(['3', '.', 's', 'n', '7', '1', 'G', '-', 'S', 'l', 'A', 'C', 'W', 't', 'M', ',', 'Ġs', 'Ġp', 'Ġm', 'Ġt', 'Ġa', 'ĠS', 'ĠC', 'Ġ1', 'ĠA', 'st', 'ĠM', 'Ġin', 'ĠP', 'Ġh', 'th', 'ĠB', 'ĠT', 'ĠR', 'Ġ19', 'ĠF', 'ĠD', 'ĠG', 'ĠW', 'ĠN', 'ĠH', 'ĠL', 'Ġ2', 'ĠI', 'ĠJ', 'Ġan', 'Ġ18', 'Ġrep', 'ĠSt', 'ĠK', 'Ġdi', 'ĠU', 'Ġdes', 'Ġ3', 'Ġper', 'Ġpre', 'Ġman', 'ĠAn', 'Ġ5', 'Ġfor', 'Ġ4', 'Ġgo', 'Ġ7', 'Ġme', 'Ġfin', 'Ġam', 'Ġ20', 'Ġy', 'Ġor', 'Ġbe', 'Ġwe', 'Ġ15', 'Ġ200', 'Ġplay', 'Ġnew', 'Ġsub', 'Ġhigh', 'Ġso', 'ĠIn', 'Ġwin', 'Ġ8', 'ĠCol', 'Ġnot', 'Ġup', 'Ġpost', 'Ġ189', 'Ġstar', 'Ġ9', 'Ġ191', 'Ġpen', 'Ġwith', 'Ġcritic', 'Ġhe', 'ĠMa', 'Ġpass', 'Ġ6', 'Ġcal', 'Ġfun', 'Ġno', 'Ġas', 'ĠPlay', 'Ġover', 'Ġunder', 'Ġdesign', 'Ġthem', 'Ġform', 'Ġ27', 'ĠLe', 'Ġsign', 'ĠAll', 'Ġnorth', 'ĠChrist', 'Ġback', 'Ġlit', 'Ġto', 'Ġdevelop', 'Ġwork', 'Ġhand', 'Ġmet', 'Ġdown', 'Ġlow', 'Ġoff', 'Ġcamp', 'Ġperson', 'Ġat', 'Ġout', 'Ġremain', 'Ġdo', 'Ġrecord', 'ĠAt', 'Ġown', 'Ġtow', 'Ġend', 'Ġlist', 'Ġpop', 'Ġengine', 'Ġpub

We provide the tokenize function to you

In [113]:
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    splits = [[l for l in word] for word in pre_tokenized_text]
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split

    return sum(splits, [])
tokenize("Darmstadt holds the official title City of Science (German: Wissenschaftsstadt) as it is a major centre of scientific institutions, universities, and high-technology companies.")

['D',
 'ar',
 'm',
 'st',
 'a',
 'd',
 't',
 'Ġh',
 'o',
 'l',
 'd',
 's',
 'Ġt',
 'h',
 'e',
 'Ġ',
 'o',
 'f',
 'f',
 'ic',
 'i',
 'al',
 'Ġ',
 'ti',
 't',
 'le',
 'ĠC',
 'it',
 'y',
 'Ġ',
 'o',
 'f',
 'ĠS',
 'c',
 'i',
 'en',
 'c',
 'e',
 'Ġ',
 '(',
 'G',
 'er',
 'm',
 'an',
 ':',
 'Ġ',
 'W',
 'is',
 's',
 'en',
 's',
 'c',
 'h',
 'a',
 'f',
 't',
 's',
 'st',
 'a',
 'd',
 't',
 ')',
 'Ġa',
 's',
 'Ġ',
 'it',
 'Ġ',
 'is',
 'Ġa',
 'Ġm',
 'a',
 'j',
 'or',
 'Ġc',
 'ent',
 're',
 'Ġ',
 'o',
 'f',
 'Ġs',
 'c',
 'i',
 'en',
 'ti',
 'f',
 'ic',
 'Ġin',
 's',
 'ti',
 't',
 'u',
 'ti',
 'on',
 's',
 ',',
 'Ġ',
 'u',
 'n',
 'i',
 'v',
 'ers',
 'i',
 'ti',
 'es',
 ',',
 'Ġ',
 'an',
 'd',
 'Ġh',
 'i',
 'g',
 'h',
 '-',
 't',
 'ec',
 'h',
 'n',
 'o',
 'lo',
 'g',
 'y',
 'Ġc',
 'om',
 'p',
 'an',
 'i',
 'es',
 '.']

### Task 1.6
Now, repeat the byte pair encoding for vocab sizes of 1000 and 5000. What do you notice?

In [114]:
# Use this chunk for the answer
vocab_size = 1000
merges = {("i", "n"): "in"}
vocab.append("in")
while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits = merge_pair(*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

In [115]:
len(merges)

800

In [116]:
vocab

['<|endoftext|>',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '=',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '~',
 '¡',
 '¢',
 '£',
 '¤',
 '¥',
 '¦',
 '©',
 'ª',
 '«',
 '®',
 '¯',
 '°',
 '±',
 '²',
 '³',
 '´',
 'µ',
 '¹',
 '¼',
 '½',
 '¿',
 'Â',
 'Ã',
 'Å',
 'â',
 'ã',
 'ä',
 'å',
 'æ',
 'ç',
 'è',
 'é',
 'ï',
 'Ċ',
 'Ġ',
 'Ģ',
 'ģ',
 'Ĥ',
 'ĥ',
 'Ħ',
 'ħ',
 'Ĩ',
 'Ī',
 'ī',
 'Ĭ',
 'ĭ',
 'Į',
 'į',
 'ı',
 'Ĳ',
 'ĳ',
 'Ĵ',
 'ĵ',
 'Ķ',
 'Ĺ',
 'ĺ',
 'Ļ',
 'Ľ',
 'Ł',
 'ł',
 'Ń',
 'in',
 'er',
 'ed',
 'on',
 'es',
 'en',
 'Ġs',
 'at',
 'an',
 'al',
 'or',
 'ar',
 'ing',
 'Ġc',
 'Ġp',
 're',
 'ti',
 'is

In [117]:
tokenize("Darmstadt holds the official title City of Science (German: Wissenschaftsstadt) as it is a major centre of scientific institutions, universities, and high-technology companies.")

['D',
 'a',
 'r',
 'm',
 's',
 't',
 'ad',
 't',
 'Ġ',
 'h',
 'old',
 's',
 'Ġ',
 'th',
 'e',
 'Ġoff',
 'i',
 'ci',
 'a',
 'l',
 'Ġ',
 't',
 'i',
 't',
 'l',
 'e',
 'Ġ',
 'C',
 'i',
 'ty',
 'Ġo',
 'f',
 'Ġ',
 'S',
 'ci',
 'e',
 'n',
 'ce',
 'Ġ',
 '(',
 'G',
 'e',
 'r',
 'm',
 'a',
 'n',
 ':',
 'ĠW',
 'i',
 'ss',
 'e',
 'n',
 's',
 'ch',
 'a',
 'f',
 'ts',
 's',
 't',
 'ad',
 't',
 ')',
 'Ġ',
 'a',
 's',
 'Ġ',
 'i',
 't',
 'Ġ',
 'i',
 's',
 'Ġ',
 'a',
 'Ġ',
 'm',
 'a',
 'j',
 'o',
 'r',
 'Ġ',
 'ce',
 'n',
 't',
 'r',
 'e',
 'Ġo',
 'f',
 'Ġ',
 's',
 'ci',
 'e',
 'n',
 't',
 'if',
 'i',
 'c',
 'Ġ',
 'ins',
 't',
 'i',
 't',
 'ut',
 'i',
 'o',
 'n',
 's',
 ',',
 'Ġun',
 'ive',
 'r',
 's',
 'i',
 't',
 'ie',
 's',
 ',',
 'Ġ',
 'a',
 'n',
 'd',
 'Ġ',
 'h',
 'igh',
 '-',
 'te',
 'ch',
 'n',
 'ol',
 'og',
 'y',
 'Ġ',
 'c',
 'o',
 'mp',
 'a',
 'n',
 'ie',
 's',
 '.']

In [118]:
# Use this chunk for the answer
vocab_size = 5000
merges = {("i", "n"): "in"}
vocab.append("in")
while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits = merge_pair(*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

In [119]:
len(merges)

4000

In [120]:
merges

{('i', 'n'): 'in',
 ('Ġin', 'sp'): 'Ġinsp',
 ('enc', 'ed'): 'enced',
 ('en', 'ed'): 'ened',
 ('Ġt', 'el'): 'Ġtel',
 ('anc', 'es'): 'ances',
 ('hi', 'b'): 'hib',
 ('Ġar', 'ti'): 'Ġarti',
 ('Ġin', 'st'): 'Ġinst',
 ('om', 'm'): 'omm',
 ('Ġ19', '9'): 'Ġ199',
 ('ĠC', 'l'): 'ĠCl',
 ('Ġsu', 'p'): 'Ġsup',
 ('ac', 'tions'): 'actions',
 ('ĠG', 'o'): 'ĠGo',
 ('Ġs', 'ix'): 'Ġsix',
 ('ous', 'ly'): 'ously',
 ('le', 'y'): 'ley',
 ('ri', 'g'): 'rig',
 ('ĠB', 'e'): 'ĠBe',
 ('an', 'y'): 'any',
 ('Ġd', 'eter'): 'Ġdeter',
 ('Ġw', 'a'): 'Ġwa',
 ('Ġauth', 'or'): 'Ġauthor',
 ('Ġf', 'u'): 'Ġfu',
 ('Ġdis', 'p'): 'Ġdisp',
 ('f', 'or'): 'for',
 ('Ġb', 'el'): 'Ġbel',
 ('Ġs', 'urr'): 'Ġsurr',
 ('ĠP', 'er'): 'ĠPer',
 ('Ġm', 'ar'): 'Ġmar',
 ('an', 'ti'): 'anti',
 ('ab', 'ric'): 'abric',
 ('Ġrep', 'air'): 'Ġrepair',
 ('ĠCon', 'f'): 'ĠConf',
 ('ĠConf', 'eder'): 'ĠConfeder',
 ('us', 's'): 'uss',
 ('Ġcon', 'ver'): 'Ġconver',
 ('Ġsk', 'et'): 'Ġsket',
 ('o', 'le'): 'ole',
 ('Ġl', 'abor'): 'Ġlabor',
 ('lo', 'y'): 'loy',
 (

In [121]:
tokenize("Darmstadt holds the official title City of Science (German: Wissenschaftsstadt) as it is a major centre of scientific institutions, universities, and high-technology companies.")

['D',
 'a',
 'r',
 'm',
 's',
 't',
 'a',
 'd',
 't',
 'Ġ',
 'h',
 'o',
 'l',
 'd',
 's',
 'Ġ',
 't',
 'he',
 'Ġ',
 'o',
 'f',
 'f',
 'i',
 'c',
 'i',
 'a',
 'l',
 'Ġ',
 't',
 'i',
 't',
 'l',
 'e',
 'Ġ',
 'C',
 'i',
 't',
 'y',
 'Ġ',
 'o',
 'f',
 'Ġ',
 'S',
 'c',
 'i',
 'e',
 'n',
 'c',
 'e',
 'Ġ(',
 'G',
 'e',
 'r',
 'm',
 'a',
 'n',
 ':',
 'Ġ',
 'W',
 'i',
 's',
 's',
 'e',
 'n',
 's',
 'c',
 'h',
 'a',
 'f',
 't',
 's',
 's',
 't',
 'a',
 'd',
 't',
 ')',
 'Ġ',
 'a',
 's',
 'Ġ',
 'i',
 't',
 'Ġ',
 'i',
 's',
 'Ġ',
 'a',
 'Ġ',
 'm',
 'a',
 'j',
 'o',
 'r',
 'Ġ',
 'c',
 'e',
 'n',
 't',
 'r',
 'e',
 'Ġ',
 'o',
 'f',
 'Ġ',
 's',
 'c',
 'i',
 'e',
 'n',
 't',
 'i',
 'f',
 'i',
 'c',
 'Ġ',
 'in',
 's',
 't',
 'i',
 't',
 'u',
 't',
 'io',
 'n',
 's',
 ',',
 'Ġu',
 'n',
 'i',
 'v',
 'e',
 'r',
 's',
 'i',
 't',
 'i',
 'e',
 's',
 ',',
 'Ġ',
 'a',
 'nd',
 'Ġ',
 'h',
 'i',
 'g',
 'h',
 '-',
 't',
 'e',
 'c',
 'h',
 'n',
 'o',
 'l',
 'o',
 'g',
 'y',
 'Ġ',
 'co',
 'm',
 'p',
 'a',
 'n',
 'i

# **Task 2: Getting to Know Foundation Models**

In this task, we want to load two different versions of GPT-2 and compare their performance.

### Task 2.1

After we imported transformers from Huggingface, your first task is to initialize the GPT2LMHead Model. This model should be a gpt2 model from huggingface under the variable model. Also define the GPT2 tokenizer.

While we have seen the transformers in the lectures, you should print a single GPT2Block to the console for the 10th layer. Then, count the parameters of the model.

In [132]:
# initialize model and tokenizer
model_small = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer_gpt2 = GPT2Tokenizer.from_pretrained("gpt2")

In [133]:
len(list(model_small.parameters()))

148

In [134]:
count_all = 0
count_trainable = 0
for p in model_small.parameters():
  count_all += p.numel()
  if p.requires_grad:
    count_trainable += p.numel()

In [135]:
for p in model_small.parameters():
  print(p.numel())
  break

38597376


In [136]:
print(count_all)
print(count_trainable)

124439808
124439808


In [137]:
total_trainable_params = sum(p.numel() for p in model_small.parameters() if p.requires_grad)
print(total_trainable_params)

124439808


In [138]:
# Print a GPT2Block in the 10th layer
# There are 12 layers, the index begins at 0, there the 10th layer has an index of 9
print(f"10th layer of GPT2LMHead: \n{model_small.transformer.h[9]}")

def count_parameters(model):
    #YOUR CODE HERE
    #trainable are equal to total in this case
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# GPT2_small parameter trainable parameters:
print(f"The model has {count_parameters(model_small):,} trainable parameters")

10th layer of GPT2LMHead: 
GPT2Block(
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (attn): GPT2Attention(
    (c_attn): Conv1D()
    (c_proj): Conv1D()
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (mlp): GPT2MLP(
    (c_fc): Conv1D()
    (c_proj): Conv1D()
    (act): NewGELUActivation()
    (dropout): Dropout(p=0.1, inplace=False)
  )
)
The model has 124,439,808 trainable parameters


In [139]:
tokenizer_gpt2("hello, this is a quick test! Do not worry.", return_tensors="pt")

{'input_ids': tensor([[31373,    11,   428,   318,   257,  2068,  1332,     0,  2141,   407,
          5490,    13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [140]:
tokenizer_gpt2.tokenize("hello, this is a quick test! Do not worry.")

['hello',
 ',',
 'Ġthis',
 'Ġis',
 'Ġa',
 'Ġquick',
 'Ġtest',
 '!',
 'ĠDo',
 'Ġnot',
 'Ġworry',
 '.']

### Task 2.2
Because we want to compare two versions of GPT, you now also load the gpt2-large from huggingface as model. As evaluation dataset you import the dataset "wikitext-2-raw-v1" from huggingface. (like in Task1) Make sure to use the 200 first entries of the test dataset and print an entry. Also print the parameters of the gpt2-large model

In [141]:
# Load the gpt2-large model from huggingface
model_large = GPT2LMHeadModel.from_pretrained("gpt2-large")
tokenizer_gpt2_large = GPT2Tokenizer.from_pretrained("gpt2-large")

In [37]:
# Load the dataset
whole_dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
test_dataset = whole_dataset["test"]["text"][:200]
# Print any single entry
print(f'Second entry: {test_dataset[1]}')
# Print Trainable parameters of huggingface gpt2-large
print(f"GPT2-LARGE Trainable Parameters: {count_parameters(model_large)}")

  0%|          | 0/3 [00:00<?, ?it/s]

Second entry:  = Robert Boulter = 

GPT2-LARGE Trainable Parameters: 774030080


### Task 2.3
As a next step, we define an evaluation metric to calculate predictions. Remember the perplexity funciton from the lecture. The function should take a model, tokenizer and a text and return the perplexity score for the given text.

Hint: you should encode the text, generate outputs for inputs and corresponding labels and then calculate perplexity.

In [186]:
# useful article: https://medium.com/@priyankads/perplexity-of-language-models-41160427ed72
def calculate_perplexity(model, tokenizer, text):
    # YOUR CODE HERE
    inputs = tokenizer(text, return_tensors="pt")
    loss = model(input_ids=inputs["input_ids"], labels=inputs["input_ids"]).loss
    ppl = torch.exp(loss)
    #only return the loss value without the backpropagation item
    return ppl.item()

In [188]:
#test
print(calculate_perplexity(model=model_large, tokenizer=tokenizer_gpt2_large, text="I went to the shop yesterday and it was sunny."))
print(calculate_perplexity(model=model_small, tokenizer=tokenizer_gpt2, text="I went to the shop yesterday and it was sunny."))

37.615272521972656
33.04161071777344


In [190]:
import numpy as np

perplexities_small = []
perplexities_large = []
# Loop over the dataset, append scores to the lists
for idx, sent in enumerate(test_dataset):
  try:
    perplexities_large.append(calculate_perplexity(model=model_large, tokenizer=tokenizer_gpt2_large, text=sent))
  except Exception as e:
        #print(f"Error occurred for sentence at index {idx}: {sent}")
        #print(f"Error message: {str(e)}")
        perplexities_large.append(np.nan)

  try:
    perplexities_small.append(calculate_perplexity(model=model_small, tokenizer=tokenizer_gpt2, text=sent))
  except Exception as e:
        #print(f"Error occurred for sentence at index {idx}: {sent}")
        #print(f"Error message: {str(e)}")
        perplexities_small.append(np.nan)

  if idx % 10 == 0:
    print(f"Step {idx} is DONE")

# Calculate and print average perplexities
print("Average perplexity (GPT2 ): ", np.nanmean(perplexities_small))
print("Average perplexity (GPT2 large): ", np.nanmean(perplexities_large))

Step 0 is DONE
Step 10 is DONE
Step 20 is DONE
Step 30 is DONE
Step 40 is DONE
Step 50 is DONE
Step 60 is DONE
Step 70 is DONE
Step 80 is DONE
Step 90 is DONE
Step 100 is DONE
Step 110 is DONE
Step 120 is DONE
Step 130 is DONE
Step 140 is DONE
Step 150 is DONE
Step 160 is DONE
Step 170 is DONE
Step 180 is DONE
Step 190 is DONE
Average perplexity (GPT2 ):  509.78487387011126
Average perplexity (GPT2 large):  433.09436268960275


In [182]:
perplexities_small[:10]

[1331.8509521484375,
 29.471433639526367,
 27.237403869628906,
 556.5676879882812,
 116.3771743774414,
 30.246723175048828,
 39.347660064697266,
 109.5719985961914,
 30.84715461730957,
 33.45499801635742]

In [191]:
perplexities_small == perplexities_large

False

What do you notice? Please explain.

### Task 2.4 Masking attention
What is an attention mask? When and why is it usually used?

Your answer here:

Your task is now to run a prompt in regular inference, but with an attention mask. You can reuse the gpt2_small model

In [ ]:
text_prompt = "The sun was setting behind"

# Step 1: Implement attention masking
def apply_attention_mask(input_ids, mask_start_idx, mask_end_idx):
    # -----------------------
    #YOUR CODE HERE
    # -----------------------
    attention_mask = torch.ones(input_ids.shape)
    # mask the given positions
    attention_mask[:, mask_start_idx : mask_end_idx + 1] = 0
    return attention_mask

# Step 2: Apply attention masks
mask_start = 3  # Start index of the region to mask
mask_end = 5  # End index of the region to mask
tokenizer_small = AutoTokenizer.from_pretrained("gpt2")
input_ids = tokenizer_small.encode(text_prompt, return_tensors='pt')
print(f"input_ids == {input_ids}")
attention_mask = apply_attention_mask(input_ids, mask_start, mask_end)

# Step 3: Generate output using modified attention mechanism
with torch.no_grad():
    outputs = model_small.generate(input_ids, attention_mask=attention_mask)
    print(f"Outputs == {outputs}")
    generated_text = tokenizer_small.decode(outputs[0], skip_special_tokens=True)

# Step 4: Compare and analyze the generated outputs
print("Text Prompt:", text_prompt)
print("Generated Text:", generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


input_ids == tensor([[ 464, 4252,  373, 4634, 2157]])
Outputs == tensor([[ 464, 4252,  373, 4634, 2157,  262,  691, 1517,  326,  714, 2245,  683,
           13,  198,  198,    1,   40, 1101, 7926,   11]])
Text Prompt: The sun was setting behind
Generated Text: The sun was setting behind the only thing that could stop him.

"I'm sorry,


In [ ]:
# Step 5: Run the same prompt without attention mask

# YOUR CODE HERE
with torch.no_grad():
    outputs = model_small.generate(input_ids)
    print(f"Outputs == {outputs[0]}")
    generated_text = tokenizer_small.decode(outputs[0], skip_special_tokens=True)

# Step 4: Compare and analyze the generated outputs
print("Text Prompt:", text_prompt)
print("Generated Text NO ATTENTION MASK:", generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Outputs == tensor([  464,  4252,   373,  4634,  2157,   262, 15114,    11,   290,   262,
         2344,   373, 19280,   287,   262,  4571,   286,   262,  4252,    13])
Text Prompt: The sun was setting behind
Generated Text NO ATTENTION MASK: The sun was setting behind the clouds, and the wind was blowing in the direction of the sun.


Step 6: Compare the two outputs and describe it in one to two sentences

Your answer here: